# 要約 
このJupyter Notebookは、Kaggleの「LLM 20 Questions」コンペティションに向けたエージェントの作成プロセスを示しています。具体的には、質問者と回答者の役割を持つ2つのエージェントを実装するための手法を扱っています。

### 問題の概要
「20の質問」ゲームで、高度な言語モデル（LLM）を活用し、質問を通じてターゲットワードを特定することが求められます。このゲームの目的は、各チームの質問者ができるだけ少ない質問でターゲットワードを推測することです。

### 使用されている手法とライブラリ
- **エージェントの実装**: `GemmaQuestionerAgent`と`GemmaAnswererAgent`がそれぞれ質問を生成し、回答を提供するエージェントです。これにより、ゲームの規則やフローに沿った対応を行います。
- **Gemmaライブラリ**: Googleの`gemma_pytorch`リポジトリからクローンしたライブラリを使用します。これは、指定された設定でのモデルの初期化やトークン化を支援します。
- **必要なパッケージ**: `immutabledict`と`sentencepiece`がインストールされており、特に不変辞書形式とトークナイザーとして機能します。
- **PyTorch**: モデルのトレーニングや推論に使用され、`torch`ライブラリを通じてTensor操作が行われます。

### プロセスの流れ
1. **環境設定**: 必要なライブラリをインストールし、ゲームプレイのために必要なファイルを適切なディレクトリに配置します。
2. **セッションの開始**: 質問者または回答者エージェントのセッションが開始され、ゲーム状態に基づいてフォーマットされたプロンプトを生成します。
3. **応答の生成**: エージェントがプロンプトを使用して言語モデルからの応答を受け取り、それを解析して次のアクションを決定します。
4. **提出ファイルの作成**: 最終的に、すべての必要なファイルを圧縮し、コンペティションへの提出用のアーカイブファイルを作成します。

このノートブックは、エージェントの役割を明確に分割し、各エージェントがゲーム内で機能するための具体的な構造を提供しています。また、モデルがどのように操作され、互いに作用するかに焦点を当てています。最終的に生成された`submission.tar.gz`ファイルをコンペティションに提出することで、エージェントのパフォーマンスを測定します。

---


# 用語概説 
以下に、Jupyter Notebookの内容を踏まえた初心者がつまずきそうな専門用語の解説を示します。

1. **immutabledict**:
   - 不変の辞書を提供するPythonのライブラリです。一度作成されると内容を変更できず、設定や定数を誤って変更してしまうのを防ぐために使われます。データが変更されることが予期されない場合に便利です。

2. **sentencepiece**:
   - Googleが開発したトークナイザーで、テキストをサブワード単位に分割します。これにより、未知の単語への対応が容易になり、特に多言語や事前学習モデルでよく利用されます。通常の単語分割器に比べ、より柔軟で効率的なトークナイゼーションを実現します。

3. **contextlib**:
   - コンテクスト管理を行うためのモジュールで、with文を使ってリソース管理を効率的に行えます。特定の処理を行った後、自動的に後処理を実行するために使用される場合が多いです。

4. **torch.dtype**:
   - PyTorchにおけるデータ型の表現です。例えば、`torch.float`, `torch.int`などがあり、テンソルのデータ型を指定するために使われます。効率的なメモリの使用や計算性能の最適化につながります。

5. **quant**:
   - モデルの重みや出力を整数や小数の固定ビット数に圧縮して、メモリ使用量を削減する技術です。量子化は特に、デプロイ時のパフォーマンス向上やリソース制約のある環境（例：モバイルデバイス）で重要です。

6. **interleave_unequal**:
   - 二つの異なる長さのリストを交差させる関数です。長さが異なる場合、短い方のリストの不足分には`None`で埋めることで、両者を交互に並べる処理を行います。データ処理やコレクション操作において役立つ技術です。

7. **@contextlib.contextmanager**:
   - Pythonのデコレーターで、関数をコンテクストマネージャとして使用できるようにします。これにより、リソースを準備し、使用後にクリーンアップする処理を自動化できます。特に、ファイル操作やネットワークリソース管理で便利です。

8. **GemmaForCausalLM**:
   - Gemmaモデルの因果言語モデル（Causal Language Model）に関するクラスで、シーケンスデータに基づいて次に来るトークンを予測するために訓練されています。このクラスは、特定のタスクに応じてカスタマイズされたモデルを提供します。

9. **load_weights**:
   - 事前に訓練されたモデルの重みを読み込むメソッドです。これにより、モデルが知識をもとに新しいデータに対して効果的に応答できるようになります。

10. **max_new_tokens**:
    - モデルから生成される新しいトークンの最大数を指定するパラメータです。この設定によって出力の長さを制御し、必要に応じて短いまたは長い応答を生成することができます。

以上の用語は、ノートブック内で使われている特有のコンセプトや一般的ではないが知識として重要なものを中心に選びました。これらを理解することで、ノートブック全体の流れや機能が把握しやすくなるでしょう。

---


[スターターノートブック](https://www.kaggle.com/code/ryanholbrook/llm-20-questions-starter-notebook)を進めながら、友人のChatGPTと共同でこの完全にドキュメント化されたバージョンを作成しました。コードは100%同じで変更はありませんが、コメントが役に立つことを願っています。

このノートブックは、**LLM 20 Questions**のエージェント作成プロセスを示しています。このノートブックを実行すると、`submission.tar.gz`ファイルが生成されます。このファイルは、右側の**コンペティションに提出**という見出しから直接提出できます。あるいは、ノートブックのビューアから*Output*タブをクリックし、`submission.tar.gz`を見つけてダウンロードします。コンペティションホームページの左上にある**エージェントを提出**をクリックして、ファイルをアップロードし、提出を行ってください。

In [ ]:
%%bash
# カレントディレクトリを作業ディレクトリに移動します
cd /kaggle/working

# immutabledictとsentencepieceライブラリを指定されたディレクトリにインストールします
pip install -q -U -t /kaggle/working/submission/lib immutabledict sentencepiece

# GitHubからgemma_pytorchリポジトリをクローンします。出力を非表示にします
git clone https://github.com/google/gemma_pytorch.git > /dev/null

# 新しいディレクトリを作成します
mkdir /kaggle/working/submission/lib/gemma/

# クローンしたリポジトリ内のgemmaのファイルを新しいディレクトリに移動します
mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/submission/lib/gemma/ 

# 上記のコマンドは、エージェントのために必要なライブラリとコードを準備しています。

上記のセルでは、必要なPythonパッケージをインストールし、エージェントで使用するために`gemma_pytorch`ライブラリを準備することで環境を整えています。これにより、すべての必要な依存関係が提出ファイルにまとめられます。

- `%%bash`はJupyterノートブックでのセルマジックコマンドであり、セル全体をbashスクリプトとして実行することを示します。つまり、`%%bash`の後のすべての行は、Pythonコードとしてではなく、シェル内でbashコマンドとして解釈され、実行されます。

- `cd /kaggle/working`は、カレントディレクトリを`/kaggle/working`に変更します。このディレクトリはKaggle環境内の作業ディレクトリであり、ファイルを保存したりワークスペースを設定したりできます。

- `pip install -q -U -t /kaggle/working/submission/lib immutabledict sentencepiece`は、`immutabledict`と`sentencepiece`というPythonパッケージをインストールします。
   - `-q`はインストールプロセスの出力を抑制します（静かなモード）。
   - `-U`は既にインストールされている場合にパッケージをアップグレードします。
   - `-t /kaggle/working/submission/lib`は、パッケージをインストールする対象ディレクトリを指定します。これにより、依存関係が提出パッケージに含まれることが保証されます。

- `git clone https://github.com/google/gemma_pytorch.git > /dev/null`は、GitHubから[`gemma_pytorch`リポジトリ](https://github.com/google/gemma_pytorch)をカレントディレクトリ（`/kaggle/working`）にクローンします。
   - `> /dev/null`は`git clone`コマンドの出力を`/dev/null`にリダイレクトし、ノートブックの出力から隠します。`/dev/null`はUnix系オペレーティングシステムにおける特別なファイルであり、データが書き込まれると破棄され、取得できません。これはあくまで出力を抑制する手段であり、ノートブックの出力をクリーンに保ち、不必要な詳細を省くことができます。

- `mkdir /kaggle/working/submission/lib/gemma/`は、`submission/lib`ディレクトリ内に`gemma`という新しいディレクトリを作成します。このディレクトリには、`gemma_pytorch`リポジトリからのファイルが格納されます。

- `mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/submission/lib/gemma/`は、クローンした`gemma_pytorch`リポジトリ内の`gemma`ディレクトリに存在するすべてのファイルを、新たに作成した`submission/lib/gemma`ディレクトリに移動します。これにより、提出パッケージに必要なファイルが含まれることが保証されます。

インストールされたパッケージは以下の通りです：

**immutabledict**は、不変の辞書を提供するパッケージです。不変の辞書は、一度作成された後に辞書が変更できないことを確実にする必要がある場合に便利です。これは、特定のアプリケーション（設定、定数、あるいは同時プログラミングの作業を行う際）でデータ構造に対する accidental な変更を防ぐのに特に重要です。

**sentencepiece**は、gemmaのトークナイザーです。

In [ ]:
%%writefile submission/main.py
# 環境設定
import os
import sys

# **重要:** コードがノートブックでもシミュレーション環境でも動作するように、システムパスを設定します。
KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if os.path.exists(KAGGLE_AGENT_PATH):
    sys.path.insert(0, os.path.join(KAGGLE_AGENT_PATH, 'lib'))
else:
    sys.path.insert(0, "/kaggle/working/submission/lib")

import contextlib
import os
import sys
from pathlib import Path

# テンソル操作とモデル設定のために必要なモジュールをインポートします
import torch
from gemma.config import get_config_for_7b, get_config_for_2b
from gemma.model import GemmaForCausalLM

# モデルの重みのパスを定義します。
# スクリプトがKaggleシミュレーション環境で実行されているかどうかを確認します。
if os.path.exists(KAGGLE_AGENT_PATH):
    # Kaggleシミュレーション環境で実行している場合、重みのパスを設定します
    WEIGHTS_PATH = os.path.join(KAGGLE_AGENT_PATH, "gemma/pytorch/7b-it-quant/2")
else:
    # ローカルまたは別の環境で実行している場合、重みのためのローカル入力ディレクトリを使用します
    WEIGHTS_PATH = "/kaggle/input/gemma/pytorch/7b-it-quant/2"

# プロンプトのフォーマット
import itertools
from typing import Iterable


class GemmaFormatter:
    """
    20質問ゲーム用のプロンプトと応答をフォーマットするためのクラス。
    """
    _start_token = '<start_of_turn>'
    _end_token = '<end_of_turn>'

    def __init__(self, system_prompt: str = None, few_shot_examples: Iterable = None):
        """
        システムプロンプトと少数ショットの例（オプション）でGemmaFormatterを初期化します。
        
        引数:
            system_prompt (str): システム用の初期プロンプト。
            few_shot_examples (Iterable): プロンプトを初期化するための例。
        """
        self._system_prompt = system_prompt
        self._few_shot_examples = few_shot_examples
        self._turn_user = f"{self._start_token}user\n{{}}{self._end_token}\n"
        self._turn_model = f"{self._start_token}model\n{{}}{self._end_token}\n"
        self.reset()

    def __repr__(self):
        """
        プロンプトの現在の状態を返します。
        
        戻り値:
            str: 現在のフォーマット済みの状態。
        """
        return self._state

    def user(self, prompt):
        """
        現在の状態にユーザーのプロンプトを追加します。
        
        引数:
            prompt (str): ユーザーのプロンプト。
        
        戻り値:
            self: GemmaFormatterインスタンス。
        """
        self._state += self._turn_user.format(prompt)
        return self

    def model(self, prompt):
        """
        現在の状態にモデルのプロンプトを追加します。
        
        引数:
            prompt (str): モデルのプロンプト。
        
        戻り値:
            self: GemmaFormatterインスタンス。
        """
        self._state += self._turn_model.format(prompt)
        return self

    def start_user_turn(self):
        """
        新しいユーザーターンを開始します。
        
        戻り値:
            self: GemmaFormatterインスタンス。
        """
        self._state += f"{self._start_token}user\n"
        return self

    def start_model_turn(self):
        """
        新しいモデルターンを開始します。
        
        戻り値:
            self: GemmaFormatterインスタンス。
        """
        self._state += f"{self._start_token}model\n"
        return self

    def end_turn(self):
        """
        現在のターンを終了します。
        
        戻り値:
            self: GemmaFormatterインスタンス。
        """
        self._state += f"{self._end_token}\n"
        return self

    def reset(self):
        """
        フォーマッタを初期状態にリセットします。
        システムプロンプトや少数ショットの例が提供されている場合は、それらもリセットします。
        
        戻り値:
            self: GemmaFormatterインスタンス。
        """
        self._state = ""
        if self._system_prompt is not None:
            self.user(self._system_prompt)
        if self._few_shot_examples is not None:
            self.apply_turns(self._few_shot_examples, start_agent='user')
        return self

    def apply_turns(self, turns: Iterable, start_agent: str):
        """
        フォーマッタに一連のターンを適用します。
        
        引数:
            turns (Iterable): 適用するターンのシーケンス。
            start_agent (str): 最初に『user』または『model』のどちらが開始するかを指定します。
        
        戻り値:
            self: GemmaFormatterインスタンス。
        """
        formatters = [self.model, self.user] if start_agent == 'model' else [self.user, self.model]
        formatters = itertools.cycle(formatters)
        for fmt, turn in zip(formatters, turns):
            fmt(turn)
        return self


# エージェントの定義
import re


@contextlib.contextmanager
def _set_default_tensor_type(dtype: torch.dtype):
    """
    PyTorchにおけるデフォルトテンソル型を設定するためのコンテクストマネージャ。
    
    このコンテクストマネージャは、指定されたdtypeにデフォルトテンソル型を一時的に設定し、
    コンテキストを終了する際に元のデフォルトテンソル型を復元します。
    
    引数:
        dtype (torch.dtype): 設定するデフォルトテンソル型。
    
    戻り値:
        None
    """
    # デフォルトtorch dtypeを指定されたdtypeに設定します。
    torch.set_default_dtype(dtype)
    yield
    # デフォルトテンソル型をfloatに復元します
    torch.set_default_dtype(torch.float)


class GemmaAgent:
    """
    Gemma言語モデルと対話するためのエージェントの基本クラス。
    このクラスはモデルの初期化、プロンプトのフォーマット、応答生成を処理します。
    """
    def __init__(self, variant='7b-it-quant', device='cuda:0', system_prompt=None, few_shot_examples=None):
        """
        指定された設定でGemmaAgentを初期化します。
        
        引数:
            variant (str): 使用するモデルのバリアント（例： '7b-it-quant'）。
            device (str): モデルを実行するデバイス（例： 'cuda:0'はGPU）。
            system_prompt (str): システム用の初期プロンプト。
            few_shot_examples (Iterable): プロンプトを初期化するための例。
        """
        self._variant = variant
        self._device = torch.device(device)
        self.formatter = GemmaFormatter(system_prompt=system_prompt, few_shot_examples=few_shot_examples)

        print("モデルを初期化しています")
        # バリアントに基づいてモデルの設定を取得します
        model_config = get_config_for_2b() if "2b" in variant else get_config_for_7b()
        model_config.tokenizer = os.path.join(WEIGHTS_PATH, "tokenizer.model")
        model_config.quant = "quant" in variant

        # デフォルトテンソル型を設定し、モデルを初期化します
        with _set_default_tensor_type(model_config.get_dtype()):
            model = GemmaForCausalLM(model_config)
            ckpt_path = os.path.join(WEIGHTS_PATH , f'gemma-{variant}.ckpt')
            model.load_weights(ckpt_path)
            self.model = model.to(self._device).eval()

    def __call__(self, obs, *args):
        """
        新しい観察を処理し、セッションを開始してプロンプトを生成し、応答を解析します。
        
        引数:
            obs (dict): ゲーム状態情報を含む観察辞書。
            *args: 追加の引数。
        
        戻り値:
            str: モデルから生成された応答。
        """
        self._start_session(obs)
        prompt = str(self.formatter)
        response = self._call_llm(prompt)
        response = self._parse_response(response, obs)
        print(f"{response=}")
        return response

    def _start_session(self, obs: dict):
        """
        エージェントの新しいセッションを開始します。
        
        引数:
            obs (dict): ゲーム状態情報を含む観察辞書。
        
        例外:
            NotImplementedError: このメソッドはサブクラスによって実装される必要があります。
        """
        raise NotImplementedError

    def _call_llm(self, prompt, max_new_tokens=32, **sampler_kwargs):
        """
        提供されたプロンプトを使用して、言語モデルから応答を生成します。
        
        引数:
            prompt (str): 言語モデルへの入力プロンプト。
            max_new_tokens (int): 生成する新しいトークンの最大数。
            **sampler_kwargs: サンプリングプロセスのための追加のキーワード引数。
        
        戻り値:
            str: モデルから生成された応答。
        """
        if sampler_kwargs is None:
            sampler_kwargs = {
                'temperature': 0.01,
                'top_p': 0.1,
                'top_k': 1,
        }
        response = self.model.generate(
            prompt,
            device=self._device,
            output_len=max_new_tokens,
            **sampler_kwargs,
        )
        return response

    def _parse_keyword(self, response: str):
        """
        モデルの応答からキーワードを抽出します。
        
        引数:
            response (str): モデルの応答。
        
        戻り値:
            str: 抽出されたキーワード。キーワードが見つからない場合は空の文字列。
        """
        # ダブルアスタリスク（**）で囲まれたサブストリングを見つけます。
        match = re.search(r"(?<=\*\*)([^*]+)(?=\*\*)", response)
        if match is None:
            keyword = ''
        else:
            keyword = match.group().lower()
        return keyword

    def _parse_response(self, response: str, obs: dict):
        """
        観察に基づいてモデルの応答を解析します。
        
        引数:
            response (str): モデルの応答。
            obs (dict): ゲーム状態情報を含む観察辞書。
        
        例外:
            NotImplementedError: このメソッドはサブクラスによって実装される必要があります。
        """
        raise NotImplementedError


def interleave_unequal(x, y):
    """
    二つのリストxとyをインタリーブし、長さが異なる場合は欠けている値にはNoneを埋めます。
    
    この関数は二つのリストを受け取り、その要素をインタリーブします。リストの長さが異なる場合は、
    欠けている値にはNoneを使用し、最終結果からこれらのNone値を除外します。

    引数:
        x (list): インタリーブする最初のリスト。
        y (list): インタリーブする二つ目のリスト。

    戻り値:
        list: xとyの要素がインタリーブされたリスト（None値は除外）。
    
    例:
        >>> interleave_unequal([1, 2, 3], ['a', 'b'])
        [1, 'a', 2, 'b', 3]
    """
    return [
        item for pair in itertools.zip_longest(x, y) for item in pair if item is not None
    ]


class GemmaQuestionerAgent(GemmaAgent):
    """
    20質問ゲームでの質問者の役割を果たすエージェント。
    
    このエージェントはゲーム状態に基づいて質問を生成し、
    言語モデルからの応答を解析して質問プロセスを続行します。
    """
    def __init__(self, *args, **kwargs):
        """
        提供された引数でGemmaQuestionerAgentを初期化します。
        
        引数:
            *args: 基底クラス初期化子に渡す位置引数。
            **kwargs: 基底クラス初期化子に渡すキーワード引数。
        """
        super().__init__(*args, **kwargs)

    def _start_session(self, obs):
        """
        質問者エージェントの新しいセッションを開始します。
        フォーマッタをリセットし、初期プロンプトと前のターンを適用します。
        
        引数:
            obs (dict): ゲーム状態情報を含む観察辞書。
                - obs.questions (list): 以前に行った質問のリスト。
                - obs.answers (list): 受け取った回答のリスト。
                - obs.turnType (str): 現在のターンのタイプ（'ask'または'guess'）。
        """
        self.formatter.reset()
        self.formatter.user("20の質問をしましょう。あなたは質問者の役割を果たします。")
        turns = interleave_unequal(obs.questions, obs.answers)
        self.formatter.apply_turns(turns, start_agent='model')
        if obs.turnType == 'ask':
            self.formatter.user("はいまたはいいえで答えられる質問をしてください。")
        elif obs.turnType == 'guess':
            self.formatter.user("今こそキーワードを推測してください。あなたの推測をダブルアスタリスクで囲んでください。")
        self.formatter.start_model_turn()

    def _parse_response(self, response: str, obs: dict):
        """
        ゲーム内のターンのタイプに基づいてモデルの応答を解析します。
        
        引数:
            response (str): 言語モデルによって生成された応答。
            obs (dict): ゲーム状態情報を含む観察辞書。
                - obs.turnType (str): 現在のターンのタイプ（'ask'または'guess'）。
        
        戻り値:
            str: ターンタイプに基づく解析された質問または推測。
        
        例外:
            ValueError: 観察内のターンタイプが不明な場合。
        """
        if obs.turnType == 'ask':
            match = re.search(".+?\?", response.replace('*', ''))
            if match is None:
                question = "それは人ですか？"
            else:
                question = match.group()
            return question
        elif obs.turnType == 'guess':
            guess = self._parse_keyword(response)
            return guess
        else:
            raise ValueError("不明なターンタイプ:", obs.turnType)


class GemmaAnswererAgent(GemmaAgent):
    """
    20質問ゲームでの回答者の役割を果たすエージェント。
    
    このエージェントはゲーム状態に基づいてはい・いいえの回答を提供し、
    言語モデルからの応答を解析して回答プロセスを続行します。
    """
    def __init__(self, *args, **kwargs):
        """
        提供された引数でGemmaAnswererAgentを初期化します。
        
        引数:
            *args: 基底クラス初期化子に渡す位置引数。
            **kwargs: 基底クラス初期化子に渡すキーワード引数。
        """
        super().__init__(*args, **kwargs)

    def _start_session(self, obs):
        """
        回答者エージェントの新しいセッションを開始します。
        フォーマッタをリセットし、初期プロンプトと以前のターンを適用します。
        
        引数:
            obs (dict): ゲーム状態情報を含む観察辞書。
                - obs.questions (list): 以前に行った質問のリスト。
                - obs.answers (list): 与えられた同様の応答のリスト。
                - obs.keyword (str): 質問者が推測しようとしているキーワード。
                - obs.category (str): キーワードのカテゴリ。
        """
        self.formatter.reset()
        self.formatter.user(f"20の質問をしましょう。あなたは回答者の役割を果たします。キーワードは{obs.keyword}で、カテゴリは{obs.category}です。")
        turns = interleave_unequal(obs.questions, obs.answers)
        self.formatter.apply_turns(turns, start_agent='user')
        self.formatter.user(f"この質問はキーワード{obs.keyword}に関するもので、カテゴリは{obs.category}です。はい・いいえで答えてください。あなたの答えをダブルアスタリスクで囲んでください（例: **はい**または**いいえ**）。")
        self.formatter.start_model_turn()

    def _parse_response(self, response: str, obs: dict):
        """
        モデルの応答を解析してはい・いいえの回答を抽出します。
        
        引数:
            response (str): 言語モデルによって生成された応答。
            obs (dict): ゲーム状態情報を含む観察辞書。
        
        戻り値:
            str: 応答に「はい」が含まれていれば'yes'、そうでなければ'no'。
        """
        answer = self._parse_keyword(response)
        return 'yes' if 'yes' in answer else 'no'


# エージェントの作成
system_prompt = "あなたは20の質問ゲームをプレイするために設計されたAIアシスタントです。このゲームでは、回答者がキーワードを思い浮かべ、質問者がはい・いいえの質問をすることで、それに答えます。キーワードは特定の人、場所、または物です。"

few_shot_examples = [
    "20の質問をしましょう。あなたは質問者の役割を果たします。最初の質問をしてください。",
    "それは人ですか？", "**いいえ**",
    "それは場所ですか？", "**はい**",
    "それは国ですか？", "**はい**今、キーワードを推測してください。",
    "**フランス**", "正解です！",
]


# **重要:** エージェントをグローバルに定義しますので、必要なエージェントだけをロードします。
# 両方をロードすると、OOM（Out Of Memory）になる可能性があります。
agent = None


def get_agent(name: str):
    """
    提供された名前に基づいて適切なエージェント（質問者または回答者）を初期化して返します。
    
    この関数はグローバル変数を使用して、エージェントのインスタンスが一度だけ作成されることを保証します。
    エージェントがまだ初期化されていない場合は、提供された名前に基づいて
    GemmaQuestionerAgentまたはGemmaAnswererAgentの新しいインスタンスを生成します。
    
    引数:
        name (str): 初期化するエージェントの名前（'questioner'または'answerer'）。
    
    戻り値:
        GemmaAgent: GemmaQuestionerAgentまたはGemmaAnswererAgentのインスタンス。
    
    例外:
        AssertionError: エージェント名が認識されないか、エージェントの初期化に失敗した場合。
    """
    global agent
    
    if agent is None and name == 'questioner':
        agent = GemmaQuestionerAgent(
            device='cuda:0',
            system_prompt=system_prompt,
            few_shot_examples=few_shot_examples,
        )
    elif agent is None and name == 'answerer':
        agent = GemmaAnswererAgent(
            device='cuda:0',
            system_prompt=system_prompt,
            few_shot_examples=few_shot_examples,
        )
    assert agent is not None, "エージェントが初期化されていません。"

    return agent


def agent_fn(obs, cfg):
    """
    ターンの種類を判定し、適切なエージェントを呼び出して応答を生成します。
    
    この関数は観察のターンタイプを調べ、それに応じて対応するエージェント
    （質問者または回答者）を利用して応答を生成します。応答がNoneまたは空である場合は「はい」を返します。
    
    引数:
        obs (dict): ゲーム状態情報を含む観察辞書。
            - obs.turnType (str): 現在のターンのタイプ（'ask'、'guess'、または'answer'）。
        cfg (dict): エージェントの設定（現在の実装では使用されません）。

    戻り値:
        str: エージェントによって生成された応答、または応答がNoneまたは空の場合は「はい」。
    """
    if obs.turnType == "ask":
        response = get_agent('questioner')(obs)
    elif obs.turnType == "guess":
        response = get_agent('questioner')(obs)
    elif obs.turnType == "answer":
        response = get_agent('answerer')(obs)
    if response is None or len(response) <= 1:
        return "はい"
    else:
        return response

In [ ]:
!apt install pigz pv > /dev/null
# 上記のコマンドは、`pigz`と`pv`という二つのパッケージをインストールします。
# `pigz`は並列圧縮ツールで、gzipの並列版です。
# `pv`はパイプビューで、データの転送速度や進捗状況を表示するための便利なツールです。
# 実行結果はノートブックの出力に表示されないようにしています。

上記のセルでは、高速圧縮のために`pigz`を、進捗監視のために`pv`をインストールします：

- **apt install pigz pv**: `pigz`（gzipの並列実装）と`pv`（Pipe Viewer、データがパイプラインを通過する進捗状況を監視するためのツール）をインストールします。
- **> /dev/null**: コマンドの出力を`/dev/null`にリダイレクトし、出力を抑制することでノートブックをクリーンに保ちます。

In [ ]:
!tar --use-compress-program='pigz --fast --recursive | pv' -cf submission.tar.gz -C /kaggle/working/submission . -C /kaggle/input/ gemma/pytorch/7b-it-quant/2
# 上記のコマンドは、指定されたディレクトリ内のファイルを圧縮して`submission.tar.gz`という名前のアーカイブファイルを作成します。
# - `--use-compress-program='pigz --fast --recursive | pv'`: 圧縮には`pigz`を使用し、進捗状況を表示するために`pv`をパイプします。
# - `-cf submission.tar.gz`: `submission.tar.gz`という名前で圧縮ファイルを作成します。
# - `-C /kaggle/working/submission .`: `/kaggle/working/submission`ディレクトリ内の内容をアーカイブに追加します。
# - `-C /kaggle/input/ gemma/pytorch/7b-it-quant/2`: 追加で`/kaggle/input/gemma/pytorch/7b-it-quant/2`ディレクトリの内容もアーカイブに追加します。

上記のセルでは、提出ディレクトリと必要なモデルファイルを圧縮タールボール（`submission.tar.gz`）にパッケージ化しています。このタールボールには以下が含まれています：
   - `/kaggle/working/submission`内のすべてのファイル。
   - `/kaggle/input`からの`gemma/pytorch/7b-it-quant/2`ディレクトリ。
   
詳細な内訳：
- **--use-compress-program='pigz --fast --recursive | pv'**: 圧縮には`pigz`を使用し、複数のCPUコアを利用して処理を高速化します。`--fast`オプションは迅速な圧縮を保証し、`--recursive`はディレクトリを再帰的に処理します。出力は`pv`を通してパイプされ、進捗状況がモニタリングされます。
- **-cf submission.tar.gz**: `submission.tar.gz`という名前のファイルを作成します。
- **-C /kaggle/working/submission**: アーカイブにファイルを追加する前に、`/kaggle/working/submission`ディレクトリに移動します。
- **.**: 現在のディレクトリ（`-C`オプションにより`/kaggle/working/submission`となります）からすべてのファイルをアーカイブに追加します。
- **-C /kaggle/input/**: さらにファイルを追加する前に、`/kaggle/input/`ディレクトリに移動します。
- **gemma/pytorch/7b-it-quant/2**: `/kaggle/input/`から`gemma/pytorch/7b-it-quant/2`ディレクトリおよびその内容をアーカイブに追加します。

In [ ]:
# 何も入力がありません。別のセルのリクエストをお願いします。